In [ ]:
#ここから実施

In [17]:
import pandas as pd
import numpy as np
import yfinance as yf

In [39]:
# 日経225のデータをyfinanceから取得（2000/1/1～2018/12/31までを取得）
n225_train = yf.download('^N225', start='2000-01-01', end='2018-12-31')

# 参考（ソフトバンクの株価を取りたいときは下記）
# sb_train = yf.download('9434.T', start='2019-01-01', end='2050-03-04')

[*********************100%***********************]  1 of 1 completed


In [31]:
n225_train

,Open,High,Low,Close,Adj Close,Volume,短期SMA,長期SMA
Date,,,,,,,,
2000-01-04,18937.449219,19187.609375,18937.449219,19002.859375,19002.859375,0,NaN,NaN
2000-01-05,19003.509766,19003.509766,18221.820312,18542.550781,18542.550781,0,NaN,NaN
2000-01-06,18574.009766,18582.740234,18168.269531,18168.269531,18168.269531,0,NaN,NaN
2000-01-07,18194.050781,18285.730469,18068.099609,18193.410156,18193.410156,0,NaN,NaN
2000-01-11,18246.099609,18887.560547,18246.099609,18850.919922,18850.919922,0,NaN,NaN
...,...,...,...,...,...,...,...,...
2020-12-24,26635.109375,26764.529297,26605.259766,26668.349609,26668.349609,47900000,25393.644850,22716.729816
2020-12-25,26708.099609,26716.609375,26638.279297,26656.609375,26656.609375,33400000,25457.917889,22733.044197
2020-12-28,26691.289062,26854.029297,26664.599609,26854.029297,26854.029297,50700000,25528.191327,22751.479946


In [33]:
def makeSma(dataDf, tankiSma, choukiSma):
    """
    関数の説明：
        yfinanceで取得してきた株価のDataFrameを基に、
        ゴールデンクロスなら買い、デッドクロスなら決済のフラグを作成してくれる関数。
    
    Args:
        dataDf: yfinanceで取得してきた株価のDataFrame
        tankiSma: 短期移動平均線の日数
        choukiSma: 長期移動平均線の日数

    Returns:
        tradesDf: 売買フラグを付与したDataFrameを返す。
    """
    # 短期移動平均線を計算
    dataDf['短期SMA'] = dataDf['Adj Close'].rolling(window=tankiSma).mean()

    # 長期移動平均線を計算
    dataDf['長期SMA'] = dataDf['Adj Close'].rolling(window=choukiSma).mean()

    # ポジションを保持するための変数
    position = 0

    #ポジションをとった時のPrice
    numPositon = 0    

    # 取引履歴を保存するDataFrame
    tradesDf = pd.DataFrame(columns=['Date', 'Position', 'Price'])

    # 移動平均線を用いたバックテストを実施
    for i in range(len(dataDf)):
        # 短期移動平均線が長期移動平均線を上抜けした場合、買いポジションを取る
        if dataDf['短期SMA'][i] > dataDf['長期SMA'][i] and position == 0:
            position = 1
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)

        # 短期移動平均線が長期移動平均線を下抜けした場合、決済
        elif dataDf['短期SMA'][i] < dataDf['長期SMA'][i] and position == 1:
            position = 0
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
    
    #決済金額を追記
    tradesDf['result']=0

    for i in range(len(tradesDf)):
        #Buyなら買う
        if tradesDf.iloc[i, 1] == 'Buy':
            numPositon = tradesDf.iloc[i, 2]
        #Sellなら決済
        if tradesDf.iloc[i, 1] == 'Sell':
            tradesDf.iloc[i, 3] = (tradesDf.iloc[i, 2] - numPositon) / numPositon

    return tradesDf



In [6]:
n225_sma = makeSma(dataDf=n225_train, tankiSma=5, choukiSma=25)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_13084\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\App

          Date Position         Price    result
0   2000-02-08      Buy  19868.880859  0.000000
1   2000-03-13     Sell  19189.929688 -0.034172
2   2000-03-24      Buy  19958.080078  0.000000
3   2000-04-18     Sell  18969.519531 -0.049532
4   2000-06-08      Buy  17004.339844  0.000000
..         ...      ...           ...       ...
235 2018-10-12     Sell  22694.660156 -0.005550
236 2018-11-12      Buy  22269.880859  0.000000
237 2018-11-15     Sell  21803.619141 -0.020937
238 2018-11-29      Buy  22262.599609  0.000000
239 2018-12-07     Sell  21678.679688 -0.026229

[240 rows x 4 columns]


In [7]:
n225_sma['result'].sum()

0.5362064095993865

In [12]:
columns = ['短期SMA', '長期SMA', '結果']
resultDf = pd.DataFrame(columns=columns)

In [35]:
resultDf.to_csv('result3.csv', index=False, encoding='cp932')



In [40]:
import warnings
warnings.filterwarnings('ignore')

#短期線
for i in range(51, 250):

    #長期線    
    for j in range(i+1, 300):
        
        n225_sma = makeSma(dataDf=n225_train, tankiSma=i, choukiSma=j)

        # 結果を保存
        new_row = {'短期SMA': i, '長期SMA': j, '結果': n225_sma['result'].sum()}
        resultDf = resultDf.append(new_row, ignore_index=True)
        
resultDf.to_csv('result.csv', index=False, encoding='cp932')

In [36]:
# 日経225のデータをyfinanceから取得
n225_test = yf.download('^N225', start='2019-01-01', end='2023-12-31')

# テストデータで検証
n225_sma = makeSma(dataDf=n225_test, tankiSma=195, choukiSma=296)

[*********************100%***********************]  1 of 1 completed


In [37]:
n225_sma['result'].sum()

0.2498771142944017

In [26]:
n225_sma.to_csv('test.csv')